In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA 
from sklearn.preprocessing import Imputer
from sklearn.metrics import euclidean_distances, mean_absolute_error
from scipy.stats import skew, norm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from itertools import repeat
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
from matplotlib_venn import venn2, venn3
from sklearn.preprocessing import LabelEncoder

import sys
sys.path.append('../gpn_test/crm/')
sys.path.append('../gpn_test/oil/')

import importlib
#import CRM_base as crm
#import fc_oil as oil

In [3]:
df_train = pd.read_csv('../../input/prepared_data/train_proc.csv',  
                       parse_dates = ['Дата'])
df_test_before = pd.read_csv('../../input/prepared_data/test_before_proc.csv',
                             parse_dates = ['Дата'])
df_test_after = pd.read_csv('../../input/prepared_data/test_after_proc.csv',
                             parse_dates = ['Дата'])

In [4]:
train_shape = df_train.shape
test_before_shape = df_test_before.shape
test_after_shape = df_test_after.shape
print('shape of train {}'.format(df_train.shape))
print('shape of test before {}'.format(df_test_before.shape))
print('shape of test after {}'.format(df_test_after.shape))

shape of train (15040, 86)
shape of test before (4156, 86)
shape of test after (240, 5)


In [5]:
lbl1 = LabelEncoder()
lbl1.fit(df_train['Характер работы'].values)
df_train['type'] = lbl1.transform(df_train['Характер работы'].values)
df_test_before['type'] = lbl1.transform(df_test_before['Характер работы'].values)

In [6]:
df_test_b_ind = df_test_before.set_index(['group', 'Скважина', 'Дата'], drop=False)
df_test_a_ind = df_test_after.set_index(['group', 'Скважина', 'Дата'], drop=False)
df_train_ind = df_train.set_index(['group', 'Скважина', 'Дата'], drop=False)

In [7]:
df_all = pd.concat((df_train, df_test_before), axis = 0)

In [8]:
df_all['days_on'] = (df_all['Время работы, ч']/24)#.fillna(0)
df_all['liq_rate'] = (df_all['Жидкость, м3']/df_all['days_on']).fillna(0)
df_all['oil_rate'] = (df_all['Нефть, м3']/df_all['days_on']).fillna(0)
df_all['inj_rate'] = (df_all['Закачка, м3']/df_all['days_on']).fillna(0)
df_all['wct'] = (1 - df_all['oil_rate']/df_all['liq_rate']).fillna(0)
df_all['well_on'] = df_all['days_on'] > 0

df_all['wof'] = (df_all['liq_rate'] - df_all['oil_rate'])/df_all['oil_rate']
df_all['oil_prod'] = df_all['oil_rate']*df_all['days_on']

df_all['oil_cum'] = df_all.groupby('Скважина')['oil_prod'].cumsum()

df_all['liq_prod'] = df_all['liq_rate']*df_all['days_on']
df_all['liq_cum'] = df_all.groupby('Скважина')['liq_prod'].cumsum()

df_all['inj_prod'] = df_all['inj_rate']*df_all['days_on']
df_all['inj_cum'] = df_all.groupby('Скважина')['inj_prod'].cumsum()

In [9]:
df_test_after['days_on'] = df_test_after['Дата'].dt.daysinmonth * 0.98
df_test_after['inj_rate'] = df_test_after['Закачка, м3']/df_test_after['days_on']
df_test_after['inj_prod'] = df_test_after['Закачка, м3']
df_test_after['inj_cum'] = df_test_after.groupby('Скважина')['inj_prod'].cumsum()

In [10]:
cols_to_use = ['Скважина', 'Дата', 'Характер работы', 'liq_rate', 'oil_rate', 'inj_rate', 'days_on', 'wct', 
               'oil_prod', 'liq_prod', 'oil_cum', 'liq_cum', 'wof', 'inj_prod', 'inj_cum', 'well_on', 'group', 'X', 'Y']
df_crm = df_all[cols_to_use].iloc[train_shape[0]:,:]
df_crm_plot = df_crm.copy().set_index(['group','Скважина', 'Дата'], drop=False)
test_groups = df_crm['group'].unique().tolist()

In [11]:
df_oil_prod_test = df_crm.loc[df_crm['Характер работы'] == 'НЕФ', ['Скважина', 'Дата', 'oil_prod']].set_index(['Дата', 'Скважина']).unstack().copy()
df_oil_prod_test.columns = df_oil_prod_test.columns.droplevel(0)

In [13]:
df_order = pd.read_csv('../../input/order3.csv')

In [15]:
mean_oil_prod =df_oil_prod_test.loc[:,df_order['well']].iloc[-12:,:].mean(axis=0)
median_oil_prod =df_oil_prod_test.loc[:,df_order['well']].iloc[-12:,:].median(axis=0)

In [16]:
subm_mean = np.zeros((12, df_order.shape[0]))
subm_median = np.zeros((12, df_order.shape[0]))
for i in range(12):
    subm_mean[i,:] = mean_oil_prod.values
    subm_median[i,:] = median_oil_prod.values

df_subm_mean = pd.DataFrame(subm_mean.T.reshape(-1,1))
df_subm_median = pd.DataFrame(subm_median.T.reshape(-1,1))

In [18]:
df_subm_mean.to_csv('subm_mean.csv')
df_subm_median.to_csv('subm_median.csv')

In [197]:
loss_oil = oil_model.get_loss(well_on_oil_train)
loss_oil(Qoil_target_train, Qoil_pred_train)

2.5105585134419974

In [196]:
mean_absolute_error(Qoil_target_train, Qoil_pred_train)

1.4032383001438593